In [71]:
import pickle
project = 'qt'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [64]:
tr_data[0]

[[0.0769336798319384, 0.06830322, 0.02726451],
 [0.35020530990483595, 0.84387445, 0.32100973],
 [0.039051982199775646, 0.10445042, 0.09972893],
 [0.08588918127386591, 0.35723928, 0.20921192],
 [0.18240670243503868, 0.4725629, 0.031362668],
 [0.048284144678094935, 0.07725294, 0.0028972698],
 [0.011965262425156062, 0.15983382, 0.1898607],
 [0.29766020138986954, 0.87091726, 0.5815161],
 [0.11617812835860392, 0.16620982, 0.3495638],
 [0.3963747591577801, 0.50993395, 0.34185353],
 [0.02919027206367599, 0.18241285, 0.13078481],
 [0.02394680522525277, 0.3813507, 0.11021323],
 [0.033108966024455094, 0.15947708, 0.13534446],
 [0.39715977717917744, 0.42265528, 0.08573324],
 [0.01803389074557814, 0.24566294, 0.00466662],
 [0.417187663762276, 0.70136863, 0.023882268],
 [0.08351644475190935, 0.6403872, 0.04141613],
 [0.045914430831285, 0.109618366, 0.5236753],
 [0.05821287449090139, 0.03001939, 0.4433952],
 [0.07651216157341487, 0.19342765, 0.34873074],
 [0.07453128110406308, 0.7826837, 0.64233553]

In [65]:
from optuna.integration import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight

In [66]:
sample_weights = compute_sample_weight(class_weight='balanced', y=tr_data[1])
# _weights = compute_class_weight(class_weight='balanced', classes=np.unique(tr_dataset[5]), y=tr_dataset[5])

In [67]:

lgb_train = lgb.Dataset(np.array(tr_data[0]), np.array(tr_data[1]), weight=sample_weights)

In [68]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
lgb_params = {
    'objective':'binary',
    'metric': 'auc',
    'verbosity':-1,
}
tuner_cv = lgb.LightGBMTunerCV(
    lgb_params, lgb_train,
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True),
                lgb.log_evaluation(100)],
    return_cvbooster=True,
    folds=folds
)

tuner_cv.run()

[I 2023-12-02 14:19:54,731] A new study created in memory with name: no-name-1f525709-3d78-4f0e-aed6-9430d7d5373f
feature_fraction, val_score: 0.811744:  14%|#4        | 1/7 [00:00<00:00,  7.77it/s]

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	cv_agg's valid auc: 0.811744 + 0.0124473
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.811744:  43%|####2     | 3/7 [00:00<00:00,  7.90it/s]

Early stopping, best iteration is:
[9]	cv_agg's valid auc: 0.811744 + 0.0124473
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	cv_agg's valid auc: 0.798658 + 0.014213
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.811973:  71%|#######1  | 5/7 [00:00<00:00,  7.29it/s]

...

Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid auc: 0.816323 + 0.0118269


min_child_samples, val_score: 0.816533:  80%|########  | 4/5 [00:00<00:00,  4.31it/s]

Early stopping, best iteration is:
[129]	cv_agg's valid auc: 0.816533 + 0.0118295
Training until validation scores don't improve for 10 rounds
[100]	cv_agg's valid auc: 0.816323 + 0.0118269


min_child_samples, val_score: 0.816533: 100%|##########| 5/5 [00:01<00:00,  4.37it/s]

Early stopping, best iteration is:
[129]	cv_agg's valid auc: 0.816533 + 0.0118295


In [31]:
import lightgbm as lgb
model = tuner_cv.get_best_booster()
lgb.plot_importance(model, figsize=(8,4), max_num_features=5, importance_type='gain')

TypeError: booster must be Booster or LGBMModel.

In [30]:
#openstack auc 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.5246511531353015
Best params:
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 3.5658465377646046e-07, 'lambda_l2': 0.017254549195087253, 'num_leaves': 4, 'feature_fraction': 1.0, 'bagging_fraction': 0.9668243143969556, 'bagging_freq': 1, 'min_child_samples': 25}
[0.56519782 0.84614833 0.21456144 ... 0.4854053  0.11443475 0.20948806]
0.8188157668992337
0.6251302477746175


In [45]:
#openstack auc 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.8179830922461786
Best params:
{'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 0.01754458037831149, 'lambda_l2': 3.297456196622105e-05, 'num_leaves': 2, 'feature_fraction': 1.0, 'bagging_fraction': 0.5258824582300985, 'bagging_freq': 1, 'min_child_samples': 20}
[0.68179457 0.85721752 0.18221707 ... 0.50415918 0.1448258  0.17491612]
0.817103682946357
0.6809611902310441


In [69]:
#qt auc 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.8165328036699602
Best params:
{'objective': 'binary', 'metric': 'auc', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 1.9317001406011016, 'lambda_l2': 3.094407820496894, 'num_leaves': 2, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
[0.36960416 0.50902352 0.14799592 ... 0.70303968 0.61321876 0.18555734]
0.7655688245292944
0.5847515942649286
